 # Práctica 2 de Sistemas Inteligentes.

 Clasificación de imagenes fashion-MNIST mediante AdaBoost.
 
* La forma más sencilla de clasificar objetos distribuidos en un espacio es dividir el espacio en dos partes y especificar que los objetos que quedan a un lado se van a clasificar según una clase y los objetos que quedan al otro lado según la otra clase. Esto es posible realizarlo de manera muy sencilla utilizando un umbral. Los umbrales pueden ser positivos o negativos y especifican para un determinado pixel (solo uno) el rango aceptable para el mismo. 

En este Notebook se incluye el código de Adaboost y la implementación de un clasificador débil de umbral. **Se debe implementar el clasificador de débil de hiperplanos y completar la sección de experimentación**.




Incluimos las librerias necesarias para obtener la base de datos, y realizar representaciones gráficas

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import os


In [0]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

Observamos el tamaño de la libreria cargada. Utilizaremos solo las primeras 4000 imágenes para coincidir con la parte básica de la práctica 2 desarrollada en java

In [0]:
x_train.shape,y_train.shape

En google colab (jupyter notebook) es muy fácil poder visualizar gráficas e imágenes. Vamos a ver el segundo elemento del conjunto de entrenamiento:

In [0]:
plt.imshow(x_train[2]),y_train[1]

# Algoritmo Adaboost
A continuación se presenta una implementación del algoritmo Adaboost

In [0]:
def adaboost(X,Y,T,random_trainer_iterations,weak_generator,weak_error):
    #Y array con -1 y 1 dependiendo de si es de la clase
    #X imagene que le pasas
    #random trainer numero de pruebas
    N = len(X) #Numero de imagenes de entrenamiento
    #h = np.zeros([T, 3], dtype=np.double) # conjunto de clasificadores debiles, T elementos, 3 huecos
    h = list()
    mejor = list()
    alpha = np.zeros(T, dtype=np.double) # confianza
    err = np.ones(T, dtype=np.double) * np.inf #inicializar err
    
    #Distribución inicialmente uniforme
    D = np.ones(N, dtype=np.double)/N  #inicializar el peso
    
    for i in range(T): #tipico for
        for rti in range(random_trainer_iterations): #
            h_tmp = weak_generator();
            [err_tmp,y_tmp] = weak_error(h_tmp,X,Y,D)
            if(err_tmp<err[i]):
                mejor = h_tmp
                err[i] = err_tmp
                y_t = y_tmp
            
        h.append(mejor)
        alpha[i] = 0.5 * np.log((1.0 - err[i]) / err[i])
        y_t[np.where(y_t == 0)] = -1 # y_t contiene 1 en los errores 
                                     #y -1 el los aciertos, por lo que cambiare...
        
        # .. el signo de y_t (se podria cambiar solo el de alpha)
        D = np.double(D * np.exp(-alpha[i] * -y_t))  
       
        D = D/np.sum(D)  # Normalise reassigned weights
        
    return [h,alpha]

A continuación se presenta el clasificador fuerte que se obtendrá a partir de Adaboost

In [0]:
def strong_classify_image(classifier,weak_apply, X):
    [h,alpha] = classifier
    N = len(X)
        
    fuerte = []
    for i in range(len(alpha)):
        h_res = weak_apply(h[i],X)
        if (i==0):
            fuerte = np.double(alpha[i] * h_res)
        else:
            fuerte = fuerte + np.double(alpha[i] * h_res)
        #from IPython.core.debugger import Tracer; Tracer()()
        
    f = np.sign(fuerte)
    return f
        
    

## Definición de los clasificadores débiles como umbral para un pixel de la imagen

In [0]:
def generate_weak_threshold():
    ht = np.zeros(3)
    ht[0] = np.random.randint(28*28) #Pos. Img.
    ht[1] = np.random.rand(1)*256    #Valor
    ht[2] = np.sign(np.random.rand(1)*2-1)  #Direccion umbral
    return ht;


def apply_weak_threshold(ht,X):
    k = int(ht[0])
    if(ht[-1]>0):
        t = X[:,k]>ht[1]
    else: 
        t = X[:,k]<=ht[1]
    
    #Verifico que las condiciones se cumplan en todas las
    #dimensiones (true si cumple, false si no)
    t_all = np.int16(t)
    t_all[np.where(t_all == 0)] = -1
    
    return t_all

def error_weak_threshold(ht,X,Y,D=1):
    k = int(ht[0])
    if(ht[-1]>0):
        t = X[:,k]>ht[1]
    else: 
        t = X[:,k]<=ht[1]
        
    
    #Verifico que las condiciones se cumplan en todas las dimensiones 
    # (true si cumple, false si no)
    t_all = np.int16(t)
    t_all[np.where(t_all == 0)] = -1
    
    #Obtengo el error con respecto a Y
    errv = np.int64(t_all != Y)
    errs = np.sum(errv * D)
    
    return [errs,errv]

In [0]:
ht = generate_weak_threshold()
ht

## Definición de los clasificadores débiles como plano


In [0]:
import random

def generate_weak_threshold():
    ht = list()
    h = list()
    t = 0
    for i in range(784):
      h.insert(i, (random.uniform(-1, 1)))
      t = t + h[i] * (random.uniform(0, 255))
    ht.insert(0,h)
    ht.insert(1,t)
    return ht

def apply_weak_threshold(ht,X):
    resultados = np.ones(len(X))
    #solucion = np.ones(len(X))
    
    for i in range (len(X)):
      resultado = 0
      hiper = ht[0]
      term = ht[1]
      
      resultado = np.sum(hiper*X[i])
      
      resultado = resultado - term
      resultados[i] = resultado
      
    #Verifico que las condiciones se cumplan en todas las
    #dimensiones (true si cumple, false si no)
    t_all = np.int16(resultados>0)
    t_all[np.where(t_all == 0)] = -1
    
    return t_all

def error_weak_threshold(ht,X,Y,D=1):
    resultados = np.ones(len(X))
    #solucion = np.ones(len(X))
    
    for i in range (len(X)):
      resultado = 0
      hiper = ht[0]
      term = ht[1]
      
      resultado = np.sum(hiper*X[i])
      
      resultado = resultado - term
      resultados[i] = resultado
    
    #Otra forma de hacerlo
    '''resultados = X[:,:] * ht[0]
    
    for i in range (len(X)):
      resultado = 0
      hiper = ht[0]
      term = ht[1]
      for j in range (784):
        resultado = resultado + resultados[i][j]
      
      resultado = resultado - term
      solucion[i] = resultado'''
      
    #Verifico que las condiciones se cumplan en todas las
    #dimensiones (true si cumple, false si no)
    t_all = np.int16(resultados>0)
    t_all[np.where(t_all == 0)] = -1
    
    #Obtengo el error con respecto a Y
    errv = np.int64(t_all != Y)
    errs = np.sum(errv * D)
    
    return [errs,errv]

# Entrenamiento para las primeras 1000 imágenes
Obtengo un clasificador fuerte por conjunto. En esta implementación se gastan las 10 clases de la base de datos al completo (no las 8 que se gastan en la práctica de Java), con lo que los resultados pueden variar sobre esta. Es sencillo utilizar los mismos conjuntos de la implementación java si se desea...

In [0]:
num_samples = 1000
f = []
for i in range(10):
    Y = np.zeros(num_samples)
    v_digit = y_train[0:num_samples]==i
    v_else = y_train[0:num_samples]!=i
    Y[v_digit] = 1
    Y[v_else] = -1
    X = x_train[0:num_samples].reshape(num_samples,28*28)
    
    #Obtengo el clasificador fuerte para el conjunto i
    f.append(adaboost(X,Y,30,100,generate_weak_threshold,error_weak_threshold))
    print(".")

Aplico el clasificador fuerte a todas las imágenes y obtengo el conjunto de clasificación

In [0]:
num_samples = 60000
r = []
for i in range(10):
    Y = np.zeros(num_samples)
    v_digit = y_train[0:num_samples]==i
    v_else = y_train[0:num_samples]!=i
    Y[v_digit] = 1
    Y[v_else] = -1
    X = x_train[0:num_samples].reshape(num_samples,28*28)
    r.append(strong_classify_image(f[i],apply_weak_threshold,X))
    print(".")

In [0]:
r_all = np.vstack([r[0],r[1],r[2],r[3],r[4],r[5],r[6],r[7],r[8],r[9]])
r_all.shape

Transformo el vector de -1 y 1 a conjunto

In [0]:
r_digits = []
for i in range(r_all.shape[1]):
    td = np.where(r_all[:,i]==1)
    if(td[0].size==0):
        rd = -1
    else:
        rd = td[0][0]
    r_digits.append(rd)

r_digits = np.asarray(r_digits)

In [0]:
r_digits

Y obtengo la tasa de aciertos

In [0]:
Y_d = y_train[0:num_samples]
correct_digits = Y_d == r_digits
np.sum(correct_digits)/(num_samples*1.0)

Aplico el clasificador fuerte a conjunto de imágenes del mismo tipo


In [0]:
num_samples = 60000
r = []

tipo = 9

Y = np.zeros(num_samples)
v_digit = y_train[0:num_samples]==tipo
v_else = y_train[0:num_samples]!=tipo
Y[v_digit] = 1
Y[v_else] = -1
X = x_train[0:num_samples].reshape(num_samples,28*28)


a = list()

for j in range(60000):
  resultado = 0
  if y_train[j] == tipo:
    a.insert(j,X[j])

c =np.zeros(6000)
c = np.asarray(a)

print("Hay " + str(len(a)) + " imágenes de la categoría " + str(tipo))
r.append(strong_classify_image(f[tipo],apply_weak_threshold,c))
print(".")

l = 0
r_all = np.vstack([r[0]])
r_all.shape
r_digits = []
for i in range(r_all.shape[1]):
    td = np.where(r_all[:,i]==1)
    if(td[0].size==0):
        rd = -1
    else:
        rd = td[0][0]
    r_digits.append(rd)

r_digits = np.asarray(r_digits)

l = 0
for u in range(6000):
  if r_digits[u] != -1:
    l = l + 1
print("Acierta " + str(l) + " imágenes")


np.sum(l)/(6000*1.0)


# Experimentación

Para la realización de pruebas voy a usar 1000 imágenes para entrenar y voy a ir cambiado T y A. 

Voy a usar solo 1000 imágenes para el entrenamiento porque la ejecución de los métodos de hiperplanos me tarda demasiado. 

He realizado las operaciones del método "aplicar el hiperplano" de forma vectorial para acelerar la ejecución.

Para el resultado voy a pasarle las 60000 imágenes para aplicarle dicho entrenamiento realizado.

Para las gráficas he utilizado el import "matplotlib".







# Explicación de lo añadido y modificado

*   He entrenado con 1000 imágenes en vez de 4000 a causa del tiempo tan extenso de ejecución.

*   He completado el apartado "Definición de los clasificadores débiles como plano".


*   He modificado un poco Adaboost para que h sea una lista ya que mi parte con los métodos del hiperplano combino numpy y listas y era necesario.

*  Añadido otro apartado arriba que me calcula la tasa de aciertos de aplicar a las imágenes de ese tipo un único clasificador fuerte de esa prenda en concreto.





## Hiperplano
### Obtener tasa de aciertos para un dígito

He obtenido la tasa de aciertos para cada tipo de prenda. Aplicandole las imágenes de ese tipo un único clasificador fuerte de esa prenda en concreto.

La primera gráfica muestra los diferentes porcentajes que da al entrenar con T = 30 y A = 100

La segunda gráfica muestra los diferentes porcentajes que da al entrenar con T = 100 y A = 500

In [0]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
datos = [67, 86, 38, 57, 54, 62, 26, 78, 72, 87]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Diferentes Prendas')

plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
datos = [72, 93, 56, 66, 59, 75, 37, 80, 81, 90]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Diferentes Prendas')

plt.show()

### Obtener tasa de aciertos para varios dígitos

He obtenido la tasa de aciertos para todos los tipo de prenda en total. Aplicandole a cierto número de imágenes de todo tipo, todos los clasificadores fuertes.

La gráfica muestra el aumento de porcentaje que supone aumentar T y A poco a poco.

In [0]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["10/50", "30/100", "50/200", "50/500", "100/500"]
datos = [48, 58, 63, 65, 68]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('T y A')

plt.show()

## Umbral
### Obtener tasa de aciertos para un dígito

He obtenido la tasa de aciertos para cada tipo de prenda. Aplicandole las imágenes de ese tipo un único clasificador fuerte de esa prenda en concreto.


La primera gráfica muestra los diferentes porcentajes que da al entrenar con T = 30 y A = 100

La segunda gráfica muestra los diferentes porcentajes que da al entrenar con T = 100 y A = 500

In [0]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
datos = [67, 92, 48, 67, 49, 73, 28, 76, 72, 86]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Diferentes Prendas')

plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
datos = [70, 93, 54, 69, 57, 80, 41, 80, 82, 91]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Diferentes Prendas')

plt.show()

### Obtener tasa de aciertos para varios dígitos

He obtenido la tasa de aciertos para todos los tipo de prenda en total. Aplicandole a cierto número de imágenes de todo tipo, todos los clasificadores fuertes.

La gráfica muestra el aumento de porcentaje que supone aumentar T y A poco a poco.

In [0]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
TA = ["10/50", "30/100", "50/200", "50/500", "100/500"]
datos = [50, 61, 65, 67, 69]
xx = range(1,len(datos)+1)

ax.bar(xx, datos, width=0.5, color=(1,0,0), align='center')
ax.set_xticks(xx)
ax.set_xticklabels(TA)
ax.set_ylabel('Porcentaje')
ax.set_xlabel('T y A')

plt.show()

## Comparacion hiperplano vs umbral

Podemos apreciar en las gráficas de aplicar todos los clasificadores que da mejores porcentajes entrenarlo con umbrales que con hiperplanos, aunque con los umbrales, tras hacer varias pruebas entrenando con los mismos T y A,  hay que destacar que el porcentaje varía más y es menos exacto que los hiperplanos.

La diferencia es de escaso porcentaje, sobre un 2%.

Los dos coinciden en que los tipos 2, 4 y 6 de prendas son los que más se confunden, tienden a acertar menos del 50%.

## Otras métricas

NO IMPLEMENTADO